In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
import wandb
import simpletransformers
import logging
from simpletransformers.classification import (ClassificationArgs,ClassificationModel)
from sklearn.metrics import accuracy_score

In [2]:
train=pd.read_csv("train.csv",encoding='latin-1')
datos =pd.read_csv("test.csv",encoding='latin-1')
datos.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [3]:
train_df=pd.DataFrame()
train_df['text']=train['text']
train_df['label']=train['target']
test_df=pd.DataFrame()
test_df['text']=datos['text']

In [4]:


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Create a ClassificationModel
model = ClassificationModel(
    model_type="roberta",
    model_name="roberta-base",
    use_cuda=True,
    args={'overwrite_output_dir': True}
)
# Train the model

"""train_vectors = model.encode_sentences(train_df["text"].to_list(), combine_strategy="mean")
eval_vectors = model.encode_sentences(eval_df["text"].to_list(), combine_strategy="mean")"""


- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'train_vectors = model.encode_sentences(train_df["text"].to_list(), combine_strategy="mean")\neval_vectors = model.encode_sentences(eval_df["text"].to_list(), combine_strategy="mean")'

In [5]:
"""newdf=pd.DataFrame(train_vectors)
datosdf=pd.DataFrame(eval_vectors)"""

'newdf=pd.DataFrame(train_vectors)\ndatosdf=pd.DataFrame(eval_vectors)'

In [6]:
"""newdf.to_csv('robertaTrain.csv',index=False)
datosdf.to_csv('robertaDatos.csv',index=False)"""

"newdf.to_csv('robertaTrain.csv',index=False)\ndatosdf.to_csv('robertaDatos.csv',index=False)"

In [7]:
entren, evalu = train_test_split(train_df, test_size=0.2)

In [8]:
model.train_model(entren,eval_df=evalu,acc=accuracy_score)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.


In [9]:
model.eval_model(evalu, acc=accuracy_score)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.6740089786676383, 'tp': 518, 'tn': 763, 'fp': 112, 'fn': 130, 'acc': 0.8411030860144452, 'eval_loss': 0.39844234849457966}


({'mcc': 0.6740089786676383,
  'tp': 518,
  'tn': 763,
  'fp': 112,
  'fn': 130,
  'acc': 0.8411030860144452,
  'eval_loss': 0.39844234849457966},
 array([[ 1.1570374 , -1.1581693 ],
        [-1.8400849 ,  1.7791535 ],
        [ 1.1235777 , -1.1496764 ],
        ...,
        [ 0.72168815, -0.78466666],
        [ 1.0019737 , -0.96024084],
        [-1.8311958 ,  1.796806  ]], dtype=float32),
  <simpletransformers.classification.classification_utils.InputExample at 0x21e2e07fb88>])

In [10]:
predictions, raw_outputs = model.predict(test_df['text'].values.tolist())

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


In [11]:
res=pd.DataFrame()
res['target']=predictions
res['id']=datos['id']
res.to_csv('simpleTransformers.csv',index=False)

In [12]:
"""sweep_config = {
    "method": "bayes",  # grid, random
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {
        "num_train_epochs": {"values": [2, 3, 5]},
        "learning_rate": {"min": 5e-5, "max": 4e-4},
    },
}

sweep_id = wandb.sweep(sweep_config, project="Simple Sweep")"""

Create sweep with ID: qj4m4387
Sweep URL: https://app.wandb.ai/farfanatico/Simple%20Sweep/sweeps/qj4m4387


In [13]:
"""model_args = ClassificationArgs()
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.evaluate_during_training = True
model_args.manual_seed = 4
model_args.use_multiprocessing = True
model_args.train_batch_size = 16
model_args.eval_batch_size = 8
model_args.labels_list = [0,1]
model_args.wandb_project = "Simple Sweep""""

In [14]:
"""def train():
    # Initialize a new wandb run
    wandb.init()

    # Create a TransformerModel
    model = ClassificationModel(
        "roberta",
        "roberta-base",
        use_cuda=True,
        args=model_args,
        sweep_config=wandb.config,
    )

    # Train the model
    model.train_model(entren, eval_df=evalu)

    # Evaluate the model
    model.eval_model(evalu)

    # Sync wandb
    wandb.join()


wandb.agent(sweep_id, train)"""

INFO:wandb.wandb_agent:Running runs: []
INFO:wandb.wandb_agent:Agent received command: run
INFO:wandb.wandb_agent:Agent starting run with config:
	learning_rate: 0.0003698527417325339
	num_train_epochs: 5


wandb: Agent Starting Run: k148k0fy with config:
	learning_rate: 0.0003698527417325339
	num_train_epochs: 5


INFO:wandb.wandb_agent:Running runs: ['k148k0fy']
wandb: Ctrl-c pressed. Waiting for runs to end. Press ctrl-c again to terminate them.
